In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [37]:
training_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
training_data = np.array(training_data)
m, n = training_data.shape
np.random.shuffle(training_data) # shuffle before splitting into dev and training sets

data_train = training_data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n] / 255.
_,m_train = X_train.shape

Y_train   # array of correct values for each observation

array([3, 0, 0, ..., 2, 4, 7])

In [38]:
# initialize parameters
def init():
    W1 = np.random.rand(10, 784) - 0.5   # - 0.5 to adjust values to be between -0.5 and +0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

# ReLU function returns 0 if the given value x is negative and x if the given value x is positive 
def ReLU(Z):
    return np.maximum(Z, 0)

# takes in value from output layer and returns a probability that the value is correct
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

# step 1
def f_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# since ReLU function return 0 if x is negative and x otherwise,
# its derivative shows that slope is 1 if x is positive and 0 otherwise
# therefore return true (1) if x is positive, and false (0) otherwise
def d_ReLU(Z):
    return Z > 0

# returns a column where all entries are 0 except for the index matching the correct value of the observation
def mat_one(Y):
    one_Y = np.zeros((Y.size, Y.max() + 1))
    one_Y[np.arange(Y.size), Y] = 1
    one_Y = one_Y.T  # change from row to column to match our matrix
    return one_Y

# undoes f_prop
def b_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_Y = mat_one(Y)
    dZ2 = A2 - one_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1).reshape(-1, 1)
    dZ1 = W2.T.dot(dZ2) * d_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1).reshape(-1, 1)
    return dW1, db1, dW2, db2

def update(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def predict(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    #print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init()
    for i in range(iterations):
        Z1, A1, Z2, A2 = f_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = b_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = predict(A2)
            print("Accuracy: ", get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [43]:
# accuracy on training set is 85.5%. can definitely improve by adding more layers 
# (right now we're only calculating two layers)
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration:  0
[8 3 2 ... 7 0 0] [3 0 0 ... 2 4 7]
Accuracy:  0.111
Iteration:  10
[8 3 2 ... 1 0 0] [3 0 0 ... 2 4 7]
Accuracy:  0.1985609756097561
Iteration:  20
[8 3 2 ... 1 9 0] [3 0 0 ... 2 4 7]
Accuracy:  0.28424390243902437
Iteration:  30
[8 0 2 ... 2 9 7] [3 0 0 ... 2 4 7]
Accuracy:  0.35397560975609754
Iteration:  40
[5 0 6 ... 2 9 7] [3 0 0 ... 2 4 7]
Accuracy:  0.4222439024390244
Iteration:  50
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.480390243902439
Iteration:  60
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.5299512195121951
Iteration:  70
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.5714146341463414
Iteration:  80
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.6072439024390244
Iteration:  90
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.6390487804878049
Iteration:  100
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.6648780487804878
Iteration:  110
[5 0 0 ... 2 4 7] [3 0 0 ... 2 4 7]
Accuracy:  0.6860487804878049
Iteration:  120
[5 0 0 ... 2 4 7] [3 0 